In [31]:
import numpy as np
import pandas as pd
import os
import pickle
import gc 

print(np.__version__)
import sys
print(sys.executable)

# 分布確認
import ydata_profiling as pdp

# 可視化
import matplotlib.pyplot as plt

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

# バリデーション
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# 評価指標
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# モデリング: lightgbm
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")


# matplotilbで日本語表示したい場合はこれをinstallしてインポートする
import japanize_matplotlib

1.19.2
d:\Application\Anaconda3\python.exe


ModuleNotFoundError: No module named 'numba.core.errors'